In [1]:
import pandas as pd
from matplotlib import pyplot as plt
from sklearn import preprocessing
import seaborn as sns
import datetime as dt
from statsmodels.tsa.stattools import adfuller
import numpy as np
import scipy
import time

In [2]:
start_date = '2016-01-04'
end_date = '2020-03-01'

In [3]:
exchange = "btc-usd" 

In [4]:
data1 = pd.read_csv("generated_price_data/"+exchange+"_daily.csv")

In [5]:
um = "trends" # usermetric: either wiki_pageview or trends 
ak = "Coinbase" #either article name or a keyword name
tu = "daily" #time unit

In [6]:
data2 = pd.read_csv("generated_"+um+"_data/"+ak+"_"+tu+".csv")

In [7]:
## Filter data by dates
data1.index = [pd.Timestamp(t) for t in data1['day']]
data2.index = [pd.Timestamp(t) for t in data2['day']]


d1 = data1[start_date:end_date].copy()
d2 = data2[start_date:end_date].copy()

In [8]:
d1.drop_duplicates(subset=['day'],keep='first',inplace=True)
d2.drop_duplicates(subset=['day'],keep='first',inplace=True)

In [9]:
print(d1.shape)
print(d2.shape)

(1519, 8)
(1519, 17)


In [10]:
feature_names1 = ['close']
feature_names2 = ['trend','sma10','sma21'] #first element either trend or pageview

In [11]:
features_interested = ['log_','log_diff_','diff_ratio_']

In [12]:
data_dict = dict()

lag = 14 
window_size = [14,30,60]
i=0

In [13]:
st = time.time()
for fi in features_interested:
    fn1 = [fi+fn for fn in feature_names1]
    fn2 = [fi+fn for fn in feature_names2]
    for col1 in fn1:
        for col2 in fn2:
            for w in window_size:
                for t in range(d1.shape[0]-lag-w):
                    ts_1 = pd.Series(d1.iloc[t+lag:t+lag+w][col1].values)
                    tf_1 = str(d1.iloc[t+lag]['day'])+' - '+str(d1.iloc[t+lag+w]['day'])

                    for l in range(lag+1):
                        ts_2 = pd.Series(d2.iloc[t+l:t+l+w][col2].values)
                        tf_2 = str(d2.iloc[t+l]['day'])+' - '+str(d2.iloc[t+l+w]['day'])
                        try:
                            corr = scipy.stats.pearsonr(np.float32(ts_1.values),np.float32(ts_2.values))[0]
                        except:
                            corr = np.nan

                        data_dict[i] = {"window_size":w,
                                        "lag":lag-l,
                                        "feature_1":col1,
                                        "feature_1_timeframe":tf_1,
                                        "feature_2":col2,
                                        "feature_2_timeframe":tf_2,
                                        "correlation":corr}

                        i += 1
ft = time.time()

/opt/anaconda3/lib/python3.8/site-packages/scipy/stats/stats.py:3862: RuntimeWarning: invalid value encountered in subtract
  ym = y.astype(dtype) - ymean


In [14]:
ft-st

377.18517804145813

In [18]:
data = pd.DataFrame.from_dict(data_dict, "index")
data

,window_size,lag,feature_1,feature_1_timeframe,feature_2,feature_2_timeframe,correlation
0,14,14,log_close,2016-01-18 - 2016-02-01,log_trend,2016-01-04 - 2016-01-18,-0.189828
1,14,13,log_close,2016-01-18 - 2016-02-01,log_trend,2016-01-05 - 2016-01-19,-0.152556
2,14,12,log_close,2016-01-18 - 2016-02-01,log_trend,2016-01-06 - 2016-01-20,-0.257444
3,14,11,log_close,2016-01-18 - 2016-02-01,log_trend,2016-01-07 - 2016-01-21,-0.500209
4,14,10,log_close,2016-01-18 - 2016-02-01,log_trend,2016-01-08 - 2016-01-22,-0.363640
...,...,...,...,...,...,...,...
595480,60,4,diff_ratio_close,2020-01-01 - 2020-03-01,diff_ratio_sma21,2019-12-28 - 2020-02-26,-0.148762
595481,60,3,diff_ratio_close,2020-01-01 - 2020-03-01,diff_ratio_sma21,2019-12-29 - 2020-02-27,0.023361
595482,60,2,diff_ratio_close,2020-01-01 - 2020-03-01,diff_ratio_sma21,2019-12-30 - 2020-02-28,0.015957
595483,60,1,diff_ratio_close,2020-01-01 - 2020-03-01,diff_ratio_sma21,2019-12-31 - 2020-02-29,0.125860


In [19]:
file_name = 'generated_correlations_data/'+um+'/'
file_name += ak+'_'+exchange+'.csv'

In [20]:
file_name

'generated_correlations_data/trends/Coinbase_btc-usd.csv'

In [21]:
data.to_csv(file_name,index=False)